# Thinking In React

Today we will be making:

<img src="thinking-in-react-mock.png">

And assume we have some JSON that is given to us:

In [ ]:
let json = 
[
  {category: "Sporting Goods", price: "$49.99", stocked: true, name: "Football"},
  {category: "Sporting Goods", price: "$9.99", stocked: true, name: "Baseball"},
  {category: "Sporting Goods", price: "$29.99", stocked: false, name: "Basketball"},
  {category: "Electronics", price: "$99.99", stocked: true, name: "iPod Touch"},
  {category: "Electronics", price: "$399.99", stocked: false, name: "iPhone 5"},
  {category: "Electronics", price: "$199.99", stocked: true, name: "Nexus 7"}
];

Now let's breakdown the UI into a component Hierarchy:

Question: How many different React Components would you make?

<img src="thinking-in-react-components.png" width = "400">

We have 5 components in the app:

1. `FilterableProductTable` (orange): contains the entire example
2. `SearchBar` (blue): receives all user input
3. `ProductTable` (green): displays and filters the data based on the user input above it
4. `ProductCategoryRow` (turquoise): displays heading for each category
5. `ProductRow` (red): displays a row for each product

And here is the hierarchy:

<ul>
    <li> <code>FilterableProductTable</code> </li>
    <ul>
        <li> <code>SearchBar</code></li>
        <li> <code>ProductTable</code></li>
        <ul>
            <li><code>ProductCategoryRow</code></li>
            <li><code>ProductRow</code></li>
        </ul>
    </ul>
</ul>

## Now lets build a static version in React

Just build this using these 5 components, making sure the children hierarchy is correct, and using props for now. We won't use state yet because state is reserved for interactive websites, and we are only making a static website for now.

First, make it without using the JSON provided. Just hard code all of the props and component elements. Use a table as the `ProductTable`:

In [ ]:
class FilterableProductTable extends React.Component{
  constructor(props){
      super(props);
  }
  render(){
      return (
          <div>
            <SearchBar />
            <ProductTable allProducts={this.props.products} filter={"inputFromSearchBar"}/>
          </div>
      );
  }
}
class SearchBar extends React.Component{
  constructor(props){
      super(props);
  }
  render(){
      return(
          <div>
            <input type="text" />
            <br></br>
            <input type="checkbox" value="inStockOnly"/>
            Only show products in stock
          </div>
      );
  }
}

class ProductTable extends React.Component{
  constructor(props){
      super(props);
  }

  render(){
      return(
          <table>
            <thead>
              <tr><th>Name</th><th>Price</th></tr>
            </thead>
            <tbody>
              <ProductCategoryRow category={"Sporting Goods"} />
              <ProductRow name={"Football"} price={"$49.99"} stocked={true} />
              <ProductRow name={"Baseball"} price={"$9.99"} stocked={true} />
              <ProductRow name={"Basketball"} price={"$29.99"} stocked={false} />
              <ProductCategoryRow category={"Electronics"} />
              <ProductRow name={"iPod Touch"} price={"$99.99"} stocked={true} />
              <ProductRow name={"iPhone 5"} price={"$399.99"} stocked={false} />
              <ProductRow name={"Nexus 7"} price={"$199.99"} stocked={true} />
            </tbody>
          </table>
      );
  }
}

class ProductCategoryRow extends React.Component {
  constructor(props){
      super(props);
  }

  render(){
      return(
          <tr><th>{this.props.category}</th></tr>
      );
  }
}

class ProductRow extends React.Component {
  constructor(props){
      super(props);
  }

  render(){
      if (this.props.stocked) {
        return <tr><td>{this.props.name}</td><td>{this.props.price}</td></tr>
          }
      else {
        return <tr><td style={{color: "#FF0000"}}>{this.props.name}</td><td>{this.props.price}</td></tr>
      }
  }
}

Now try to do it with it adjusting to whatever the JSON is:

1. Find all unique categories. (Hint: Use for loop and `.includes`, and make sure you `console.log` to make sure it works) 
2. Once you have all of the categories, use this array in the `ProductTable`, and find out how to list all of the Objects that correspond with the right category, and show it on screen.
    1. You need to make an array variable in the constructor of `ProductTable` that will hold all of the component elements.
    2. Use a for loop to go through each unique category and add a `ProductCategoryRow` with THAT category as the prop, to the array variable. Only through each iteration will you add one.
        * In this for loop, you will need to use ANOTHER for loop that goes through all of the JSON objects to see which object's categories matches with the current category in the outer loop. If it matches, add a `ProductRow` to the array variable you made, and make sure you pass the current object's data as props.
3. Render it in the table body.

Go back to the lesson going over "Lists and Keys" to find examples of this if confused.

This is SUPER hard, so you may need to guide them slowly.

## Identify all possibilities of UI State

What are all the possible things that can change?
* The data itself (JSON)
* The filter that the user types
* The checkbox
* The filtered list of products

Now let's figure out which of these will use "state". Ask three questions about each piece of data:

1. Is it passed in from a parent via props? If so, it probably isn't state.
2. Does it remain unchanged over time? If so, it probably isn't state.
3. Can you compute it based on any other state or props in your component? If so, it isn't state.

* The data itself is not state, because it just uses the variable that we created, or passed through a prop.
* The filter does change over time
* The checkbox does change over time
* Products also changes over time, BUT is computated through the original list of products WITH the search text and value of the checkbox.

## Identify where state should live

Note that the input from `<SearchBar />` is needed for the `<ProductTable />`. And they are both direct descendants of `<FilterableProductTable />`. This means the state needs to be IN `<FilterableProductTable />`.

So we will add `this.state = {filterText: '', inStockOnly: false}` to the constructor of `FilterableProductTable`.

Now pass these two states to the `ProductTable` as props and make code that will filter the products based on these two props:

In [ ]:
for (const obj of json){
    if (obj.category === category){
        if ((this.props.inStockOnly && obj.stocked) || !this.props.inStockOnly){
            if (this.props.filterText === '' || (obj.name.includes(this.props.filterText))){
                tryTwo.push(<ProductRow name={obj.name} price={obj.price} stocked={obj.stocked} key={count.toString()}/>)
                count++;
            }
        }
    }
}

Try to test your code out by DIRECTLY changing `this.state = {filterText: '', inStockOnly: false}`, by changing their values.

## Add Inverse data flow

This part is really hard, so let's ask questions to give you an idea of what to type, and where to type it:

1. Because of the two separate states, we are going to have two separate handlers. What should each handler look for that is changing? Create the two handlers and name them appropriately.
2. For the first handler, we are going to pass in text to it. Name the parameter accordingly. Using this, which state do we change? Write the code to change that state. Do the same thing for the second handler.
3. Here is the big questions. Let's look at only the `inStockOnly` state handler. When do we want to run this handler to set the state?
4. When we click on the checkbox, does `<SearchBar>` know the box has been clicked, or `<Filterable...>`?
5. What can we do code wise, to run the `inStockOnly` state handler that is in `<Filterable...>`, in `<SearchBar>`? (Weird English, but lets say there is a method in `<Filterable...>` named `method()`. How can we run that same method in `<SearchBar>`?)
6. What attribute can we put in the `<input>` so we know that the input has been changed?

Now look at the answers and keep going once understood.

Answers:

1. First handler looks for the filter text changing. The second will look for the stock check changing.

In [ ]:
handleFilterChange(){

}
  handleStockChange(){

}

2. We are changing the `filterText` state in the first handler, and `inStockOnly` state in the second handler.

handleFilterChange(text){
    this.setState({filterText: text});
}
  handleStockChange(inStock){
    this.setState({inStockOnly: inStock});
}

3. We want to run it when we click on the button, which is the `<input type="checkbox">` one.
4. The `<SearchBar>` knows its been clicked.
5. We must pass the handler to the `<SearchBar>` component as a prop.
6. `onChange`

Let's take a look at the final `FilterableProductTable`:

In [ ]:
class FilterableProductTable extends React.Component{
  constructor(props){
      super(props);

      this.state = {filterText: '', inStockOnly: false};
      this.handleFilterChange = this.handleFilterChange.bind(this);
      this.handleStockChange = this.handleStockChange.bind(this);
  }

  handleFilterChange(filterText){
    this.setState({filterText: filterText});
  }
  handleStockChange(inStockOnly){
    this.setState({inStockOnly: inStockOnly});
  }

  render(){
      return (
          <div>
            <SearchBar filterText={this.state.filterText} inStockOnly={this.state.inStockOnly} onFilterChange={this.handleFilterChange} onStockChange={this.handleStockChange}/>
            <ProductTable filterText={this.state.filterText} inStockOnly={this.state.inStockOnly}/>
          </div>
      );
  }
}

Notice how we passed in both handlers as props to the `<SearchBar>` component. So `<SearchBar>` is going to run this method in order to change the state in `<Filterable...>`. Also notice that we pass in the `filterText` and `inStockOnly` states, so that the `<SearchBar>` can use these values later as props.

Once this is done, take a look at the final `SearchBar` code:

In [ ]:
class SearchBar extends React.Component{
  constructor(props){
      super(props);
      this.handleFilterChange = this.handleFilterChange.bind(this);
      this.handleStockChange = this.handleStockChange.bind(this);
  }

  handleFilterChange(e){
    this.props.onFilterChange(e.target.value);
  }
  
  handleStockChange(e){
    this.props.onStockChange(e.target.checked);
  }

  render(){
      return(
          <form>
            <input type="text" value={this.props.filterText} onChange={this.handleFilterChange}/>
            <br></br>
            <input type="checkbox" checked={this.props.inStockOnly} onChange={this.handleStockChange}/>
            Only show products in stock
          </form>
      );
  }
}

Now directly coming from the last explained thing, on how `<Filterable...>` passed `filterText` and `inStockOnly`, notice how `<SearchBar>` uses it in the inputs, like `<input ... value={this.props.filterText}>` and `<input ... checked={this.props.inStockOnly}>`. This is important because of how the `value` and `checked` attributes is what will be used as parameters for the handler methods, as `e.target.value` and `e.target.checked` respectively. The `onChange` that we talked about when the user changes an input, will run the `<SearchBar>` handlers, which will run the `<Filterable...>` handlers that we passed as props, and will use the `value` and `checked` as parameters. And remember, that the `<Filterable>` handlers will set the state, and we are done :)